# Role Prompt

Asking the LLM to assume a specific role to help provide a context or perspectice for the task.

- Specify the role, e.g., "As a copywriter, ..."

In [1]:
!pip install langchain openai langchain_community python-dotenv --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 367.8/367.8 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.4/396.4 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.5 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
from dotenv import load_dotenv

load_dotenv('/content/drive/MyDrive/Active Loop RAG/api.env')

True

In [4]:
from langchain import PromptTemplate, LLMChain
from langchain.llms import OpenAI

In [5]:
llm = OpenAI(model_name='gpt-3.5-turbo-instruct', temperature=0)

<ipython-input-5-eca3edb999ea>:1: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  llm = OpenAI(model_name='gpt-3.5-turbo-instruct', temperature=0)


In [6]:
template = """
As a futuristic robot band conductor, I need you to help me come up with a song title.
What's cool song title for a song about {theme} in the year {year}?
"""

In [7]:
prompt = PromptTemplate(
    input_variables=["theme", "year"],
    template=template
)

In [8]:
input_data = {"theme": "interstellar travel", "year":"3030"}

In [9]:
chain = prompt | llm

In [10]:
response = chain.invoke(input_data)

In [11]:
print("Theme: interstellar travel")
print("Year: 3030")
print("AI-generated song title:", response)

Theme: interstellar travel
Year: 3030
AI-generated song title: 
"Galactic Odyssey: Journey to 3030"


# Few shot prompting

In [12]:
from langchain import PromptTemplate, FewShotPromptTemplate, LLMChain
from langchain.callbacks import get_openai_callback
from langchain.llms import OpenAI

In [13]:
examples = [
    {"color": "red", "emotion": "passion"},
    {"color":"blue", "emotion": "serenity"},
    {"color":"green", "emotion": "tranquility"}
]

In [14]:
example_formatter_template = """
Color: {color}
Emotion: {emotion}
"""

example_prompt = PromptTemplate(
    input_variables = ["color", "emotion"],
    template=example_formatter_template
)

In [22]:
few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="Here are some examples of colors and the emotions associated with them:\n\n",
    suffix="\n\nNow, given a new color, identify the emotion associated with it:\n\nColor:{color}\nEmotion:",
    input_variables=['color'],
    example_separator='\n'
)

In [24]:
formatted_prompt = few_shot_prompt.format(color="purple")

In [25]:
chain = few_shot_prompt | llm

In [26]:
input_data = {"color": "purple"}

In [27]:
with get_openai_callback() as cb:
  response = chain.invoke(input_data)

In [28]:
print("Color: purple")
print("Emotion:", response)
print(cb)

Color: purple
Emotion:  royalty
Tokens Used: 64
	Prompt Tokens: 63
	Completion Tokens: 1
Successful Requests: 1
Total Cost (USD): $9.65e-05


# Chain Prompting

In [61]:
from langchain import PromptTemplate, LLMChain
from langchain.llms import OpenAI
from langchain.callbacks import get_openai_callback
from  textwrap import TextWrapper, wrap

In [31]:
llm = OpenAI(model_name='gpt-3.5-turbo-instruct', temperature=0)

In [34]:
template_question = """
What is the name of the famous scientist who developed {concept}?
Answer:
"""

prompt_question = PromptTemplate(template = template_question, input_variables=['concept'])

In [50]:
template_fact = """
Provide a brief description of {scientist}'s {concept}
Answer:
"""

prompt_fact = PromptTemplate(template = template_fact, input_variables = ['scientist', 'concept'])

In [44]:
chain_question = prompt_question | llm

concept = 'the general theorey of relativity'

with get_openai_callback() as cb:
  scientist_who = chain_question.invoke('the general theorey of relativity')

In [51]:
chain_fact = prompt_fact | llm

input_data = {'scientist': str(scientist_who), 'concept': str(concept)}

with get_openai_callback() as cb:
  response = chain_fact.invoke(input_data)

In [65]:
print('\n'.join(wrap(text=response)))

The general theory of relativity, developed by Albert Einstein in
1915, is a theory of gravity that explains the relationship between
space and time. It states that massive objects, such as planets and
stars, cause a curvature in the fabric of space-time, which is
perceived as the force of gravity. This theory also predicts that the
path of light is bent by the presence of massive objects, which has
been confirmed through observations of gravitational lensing. The
general theory of relativity has been a cornerstone of modern physics
and has been crucial in understanding the behavior of the universe on
a large scale.


# Well structured few shots prompt

In [66]:
from langchain import FewShotPromptTemplate, PromptTemplate, LLMChain
from langchain.llms import OpenAI

In [67]:
llm = OpenAI(model_name='gpt-3.5-turbo-instruct', temperature=0)

In [68]:
examples = [
    {
        "query": "What is the secret to happiness?",
        "answer": "Finding balance in life and learning to enjoy the small moments."
    },
    {
        "query": "How can I become more productive?",
        "answer": "Try prioritizing tasks, setting goals, and maintaining a healthy work-life balance."
    }
]

In [70]:
example_template = """
User: {query}
AI: {answer}
"""

In [71]:
example_prompt = PromptTemplate(template=example_template, input_variables=["query", "answer"])

In [72]:
prefix = """The following are excerpts from conversations with an AI
life coach. The assistant provides insightful and practical advice to the users' questions. Here are some
examples:
"""

suffix = """
User: {query}
AI: """

In [75]:
few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator='\n\n'
)

In [76]:
chain = few_shot_prompt_template | llm

In [78]:
with get_openai_callback() as cb:
  response = chain.invoke({"query": "What are some tips for improving communication skills?"})

In [79]:
print('\n'.join(wrap(response)))

Active listening, being clear and concise, and practicing empathy are
all important skills for effective communication. Also, don't be
afraid to ask for clarification or feedback to ensure understanding.


# Dynamic prompting - LengthBasedExample

In [88]:
! pip install langchain_openai --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 28.9 MB/s eta 0:00:00


In [90]:
from langchain_openai.chat_models import ChatOpenAI

In [80]:
examples = [
    {
        "query": "How do you feel today?",
        "answer": "As an AI, I don't have feelings, but I've got jokes!"
    }, {
        "query": "What is the speed of light?",
        "answer": "Fast enough to make a round trip around Earth 7.5 times in one second!"
    }, {
        "query": "What is a quantum computer?",
        "answer": "A magical box that harnesses the power of subatomic particles to solve complex problems."
    }, {
        "query": "Who invented the telephone?",
        "answer": "Alexander Graham Bell, the original 'ringmaster'."
    }, {
        "query": "What programming language is best for AI development?",
        "answer": "Python, because it's the only snake that won't bite."
    }, {
        "query": "What is the capital of France?",
        "answer": "Paris, the city of love and baguettes."
    }, {
        "query": "What is photosynthesis?",
        "answer": "A plant's way of saying 'I'll turn this sunlight into food. You're welcome, Earth.'"
    }, {
        "query": "What is the tallest mountain on Earth?",
        "answer": "Mount Everest, Earth's most impressive bump."
    }, {
        "query": "What is the most abundant element in the universe?",
        "answer": "Hydrogen, the basic building block of cosmic smoothies."
    }, {
        "query": "What is the largest mammal on Earth?",
        "answer": "The blue whale, the original heavyweight champion of the world."
    }, {
        "query": "What is the fastest land animal?",
        "answer": "The cheetah, the ultimate sprinter of the animal kingdom."
    }, {
        "query": "What is the square root of 144?",
        "answer": "12, the number of eggs you need for a really big omelette."
    }, {
        "query": "What is the average temperature on Mars?",
        "answer": "Cold enough to make a Martian wish for a sweater and a hot cocoa."
    }
]

In [81]:
from langchain.prompts.example_selector import LengthBasedExampleSelector

example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=100
)

In [85]:
dynamic_prompt_template = FewShotPromptTemplate(
    example_selector = example_selector,
    example_prompt = example_prompt,
    prefix = prefix,
    suffix = suffix,
    input_variables = ["query"],
    example_separator="\n"
)

In [91]:
llm = ChatOpenAI(model_name='gpt-3.5-turbo')

In [92]:
chain = dynamic_prompt_template | llm

In [93]:
input_data = {"query": "Who invented the telephone?"}

response = chain.invoke(input_data)

In [95]:
print(response.content)

Alexander Graham Bell is credited with inventing the telephone.


---